In [1]:
import sqlite3
import pprint as pp
import numpy as np
from sklearn.neighbors import NearestNeighbors
import datetime

In [2]:
def readTags(path, query):  
    conn = sqlite3.connect(path)
    res = conn.execute(query)
    tags = res.fetchall()
    # pp.pprint(data)
    print "Total: ", len(tags)
    return conn, tags

In [3]:
def createTermsIndices(tags):
    tagsDict = {}
    for index in range(len(tags)):
        tagsDict[tags[index][0]] = index
    print len(tagsDict.values())
    return tagsDict

In [4]:
def getArtistTerms(artist):
    query="SELECT term FROM artist_term where artist_id='%s';" %(artist)
    res = term.execute(query)
    artistTerms = res.fetchall()
    return artistTerms

In [5]:
def Timer():
    return datetime.datetime.now()

In [6]:
def createMatrix(artists, terms):
    mat = np.zeros((len(artists),len(terms)))
    print mat.shape

    for artistIndex in range(len(artists)):
        if artistIndex%10000 == 0:
            print "Processed %d items" %(artistIndex)
        artistTerms = getArtistTerms(artists[artistIndex])
        for artistTermIndex in range(len(artistTerms)):
            if artistTerms[artistTermIndex][0] in termsDict.keys():
    #             print termsDict[artistTerms[artistTermIndex][0]], artistTerms[artistTermIndex][0]
                mat[artistIndex][(termsDict[artistTerms[artistTermIndex][0]])]=1.0
    return mat

In [7]:
def createTable(filename, tablename):
    createConn = sqlite3.connect(filename)
    createTableQuery = """ CREATE TABLE IF NOT EXISTS %s (
                                        target text,
                                        similar text
                                    ); """ %(tablename)

    createConn.execute(createTableQuery)
    return createConn

In [20]:
def insertTable(createConn, tablename, indices, knn):
    counter = 0
    for currentIndex in indices:
        counter = counter+1
        if counter%10000 == 0:
            print "Inserted %d items" %(counter)
        for index in range(1, len(currentIndex)):
            if index == knn:
                break
    #         print artists[currentIndex[0]][0], artists[currentIndex[index]][0]
            insertQuery = """ INSERT INTO %s VALUES ('%s', '%s'); """ %(tablename, artists[currentIndex[0]][0], artists[currentIndex[index]][0])
    #         print insertQuery
            createConn.execute(insertQuery)
            

In [21]:
def checkTable(createConn, tablename):
    res =  createConn.execute("Select count(*) from %s;" %(tablename))
    print res.fetchall()

def deleteTable(createConn):
    createConn.execute("drop table %s;" %(tablename))

In [10]:
term, artists = readTags('data/artist_term.db', 'SELECT artist_id FROM artists;')
term, terms = readTags('data/artist_term.db',
                        'SELECT term FROM artist_term GROUP BY term HAVING count(*)>500;')
termsDict = createTermsIndices(terms)
mat = createMatrix(artists, terms)

Total:  44745
Total:  353
353
(44745, 353)
Processed 0 items
Processed 10000 items
Processed 20000 items
Processed 30000 items
Processed 40000 items


In [12]:
start = Timer()
nbrs = NearestNeighbors(n_neighbors=200, algorithm='ball_tree').fit(mat)
distances, indices = nbrs.kneighbors(mat)
print indices
print distances
end = Timer()
print end - start

[[    0 34301 19676 ..., 23140 29070 21312]
 [    1 11189   294 ..., 35093 10822  1673]
 [    2 32635  6864 ...,  3377  8689  9922]
 ..., 
 [44742 34879 43158 ..., 12136 12319 10269]
 [10209 14681 13693 ..., 13696 13714 11131]
 [44744 36796 41754 ..., 10200   366 11607]]
[[ 0.          3.87298335  3.87298335 ...,  4.58257569  4.58257569
   4.69041576]
 [ 0.          2.          2.         ...,  2.64575131  2.64575131
   2.64575131]
 [ 0.          5.          5.38516481 ...,  6.32455532  6.32455532
   6.32455532]
 ..., 
 [ 0.          2.82842712  2.82842712 ...,  3.46410162  3.46410162
   3.46410162]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          3.          3.         ...,  3.74165739  3.74165739
   3.74165739]]
0:20:20.114662


## Calculating Accuracy

In [82]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'balltree_tags_60')
checkTable(createConn, 'balltree_tags_60')
insertTable(createConn, 'balltree_tags_60', indices, 60)
checkTable(createConn, 'balltree_tags_60')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(2639955,)]
0:00:51.022962


In [83]:
start = Timer()
learning_factor=7.3
total = 0.00
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM balltree_tags_60 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:00.705551
0:00:01.442551
0:00:02.159100
0:00:02.847782
0:00:03.507848
0:00:04.173985
0:00:04.845972
0:00:05.519046
0:00:06.313626
0:00:06.981372
0:00:07.662645
0:00:08.352913
0:00:09.033202
0:00:09.740110
0:00:10.430825
0:00:11.105322
0:00:11.777039
0:00:12.506866
0:00:13.228365
0:00:13.886113


In [84]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target LIMIT 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)
print denominator

141.0


In [85]:
print "Accuracy Score for Ball-Tree for 60 Neighbours", total/denominator
createConn.close()

Accuracy Score for Ball-Tree for 60 Neighbours 0.673758865248


In [76]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'balltree_tags_45')
checkTable(createConn, 'balltree_tags_45')
insertTable(createConn, 'balltree_tags_45', indices, 45)
checkTable(createConn, 'balltree_tags_45')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(1968780,)]
0:00:42.602151


In [77]:
start = Timer()
total = 0.00
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM balltree_tags_45 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:00.558102
0:00:01.128113
0:00:01.651828
0:00:02.175019
0:00:02.674247
0:00:03.286961
0:00:03.903490
0:00:04.494642
0:00:05.029753
0:00:05.593303
0:00:06.244183
0:00:06.785704
0:00:07.304560
0:00:07.810705
0:00:08.326918
0:00:08.845249
0:00:09.410863
0:00:09.933761
0:00:10.466468
0:00:10.975969


In [79]:
print "Accuracy Score for Ball-Tree for 45 Neighbours", total/denominator #45 neighbours algorithm: Ball_tree
createConn.close()

Accuracy Score for Ball-Tree for 45 Neighbours 0.609929078014


In [86]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'balltree_tags_90')
checkTable(createConn, 'balltree_tags_90')
insertTable(createConn, 'balltree_tags_90', indices, 90)
checkTable(createConn, 'balltree_tags_90')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(3982305,)]
0:01:14.245004


In [87]:
start = Timer()
total = 0.00
learning_factor=5.2
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM balltree_tags_90 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:01.024675
0:00:02.039585
0:00:03.055631
0:00:04.168233
0:00:05.355255
0:00:06.399954
0:00:07.522884
0:00:08.594564
0:00:09.604729
0:00:10.621946
0:00:11.629012
0:00:12.630779
0:00:13.651726
0:00:14.684730
0:00:15.712343
0:00:16.720478
0:00:17.792074
0:00:18.846301
0:00:19.884201
0:00:20.905561


In [88]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target LIMIT 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)
# print denominator

In [89]:
print "Accuracy Score for Ball-Tree for 90 Neighbours", total/denominator #90 neighbours algorithm: Ball_tree
createConn.close()

Accuracy Score for Ball-Tree for 90 Neighbours 0.567164179104
